# Prompt Templates, intro

CassIO powers a sophisticated set of bindings to seamlessly inject data
from Cassandra tables into your LangChain prompt templates.

## Basic usage

In [1]:
from langchain.prompts import createCassandraPromptTemplate

This cell simply obtains a `Session` object, i.e. an active connection to your database. Replace with custom code if you are not using Astra DB.

In [2]:
from cqlsession import getCQLSession, getCQLKeyspace
astraSession = getCQLSession()
astraKeyspace = getCQLKeyspace()

/home/stefano/.virtualenvs/langchain-cassio-3.10/lib/python3.10/site-packages/cassandra/io/asyncorereactor.py:34: DeprecationWarning: The asyncore module is deprecated and will be removed in Python 3.12. The recommended replacement is asyncio
  import asyncore
/home/stefano/.virtualenvs/langchain-cassio-3.10/lib/python3.10/site-packages/cassandra/datastax/cloud/__init__.py:173: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  ssl_context = SSLContext(PROTOCOL_TLS)
/home/stefano/.virtualenvs/langchain-cassio-3.10/lib/python3.10/site-packages/cassandra/io/asyncorereactor.py:347: DeprecationWarning: ssl.match_hostname() is deprecated
  self._connect_socket()


In [3]:
ctemplate0 = """Please answer a question from a user.
Keep in mind that the user's age is {user_age} and they live in a city with
nickname {city_nickname}.

USER'S QUESTION: {user_question}
YOUR ANSWER:
"""

### Natural binding with the DB

In the (string) template above, some variables are to be filled with a DB lookup.

The following instructions specifies the details of the binding: for instance,
the variable `user_age` is to be found on table `people`, specifically in column `age`:

In [4]:
cassPrompt = createCassandraPromptTemplate(
    session=astraSession,
    keyspace=astraKeyspace,
    template=ctemplate0,
    input_variables=['city', 'name', 'user_question'],
    field_mapper={
        'user_age': ('people', 'age'),
        'city_nickname': ('nickname_by_city', 'nickname'),
    },
)

**Note** that in the command above we specify the _primary key columns_ as `input_variables`, and not the variable names found in the prompt string above.

When formatting the Prompt Template, we will have to specify the primary key values
for the DB lookup -- the rest is done by the prompt template.

In this case there are two lookups from as many tables: the prompt template
takes care of everything, provided you pass all the primary key columns required
across tables.

In [5]:
print(cassPrompt.format(city='turin', name='beppe',
                        user_question='Is functional programming fun?'))

Please answer a question from a user.
Keep in mind that the user's age is 2 and they live in a city with
nickname CereaNeh.

USER'S QUESTION: Is functional programming fun?
YOUR ANSWER:

